In [ ]:
import pandas as pd

In [ ]:
instances = pd.read_csv('./data/cui2name.tsv', sep='\t', header=None, names=['cui', 'name'])

#results = pd.read_csv('./predictions/DISRUPTS_ComplEx_outfile.txt', sep='\t')
#results = pd.read_csv('./predictions/INHIBITS_ComplEx_outfile.txt', sep='\t')
#results = pd.read_csv('./predictions/PREVENTS_ComplEx_outfile.txt', sep='\t')
#results = pd.read_csv('./predictions/TREATS_ComplEx_outfile.txt', sep='\t')

#results = pd.read_csv('./predictions/DISRUPTS_DistMult_outfile.txt', sep='\t')
#results = pd.read_csv('./predictions/INHIBITS_DistMult_outfile.txt', sep='\t')
#results = pd.read_csv('./predictions/PREVENTS_DistMult_outfile.txt', sep='\t')
#results = pd.read_csv('./predictions/TREATS_DistMult_outfile.txt', sep='\t')

#results = pd.read_csv('./predictions/DISRUPTS_RotatE_outfile.txt', sep='\t')
#results = pd.read_csv('./predictions/INHIBITS_RotatE_outfile.txt', sep='\t')
#results = pd.read_csv('./predictions/PREVENTS_RotatE_outfile.txt', sep='\t')
results = pd.read_csv('./predictions/TREATS_RotatE_outfile.txt', sep='\t')

entities = pd.read_csv('./train/entities.tsv', sep='\t', names=['id','entity'])
relations = pd.read_csv('./train/relations.tsv', sep='\t', names=['id','rel'])

In [ ]:
(instances
 .drop_duplicates(subset=['cui'])
 .merge(entities, how='right', left_on='cui', right_on='entity')
 .to_csv('entities-cui2name.tsv.gz', sep='\t', columns=['cui', 'name'], header=False, index=False, compression='gzip')
)

In [ ]:
results.sort_values('score', ascending=False, inplace=True)

In [ ]:
results = results.iloc[3:]

In [ ]:
results = results.head(100000)

In [ ]:
id2entity = {}
for i in range(entities.shape[0]):
    id2entity[i] = entities['entity'].values[i]

id2rel = {}
for i in range(relations.shape[0]):
    id2rel[i] = relations['rel'].values[i]
    
cui2name = dict(zip(instances['cui'], instances['name']))

In [ ]:
df = results.replace({'head': id2entity, 'rel': id2rel, 'tail': id2entity})

In [ ]:
def lookup(cui):
    return cui2name[cui]

df['head_name'] = df.apply(lambda row: lookup(row['head']), axis=1)
df['tail_name'] = df.apply(lambda row: lookup(row['tail']), axis=1)

In [ ]:
dct = {}
with open('./data/classified_instances.csv') as fh:
    for line in fh:
        fields = line.rstrip('\n').split('\t')
        triple = (fields[0], fields[2], fields[3])
        if triple not in dct:
            dct[triple] = 1

In [ ]:
def is_novel(head, rel, tail):
    triple = (head, rel, tail)
    if triple in dct:
        return False
    else:
        return True

df['novel'] = df.apply(lambda row: is_novel(row['head'], row['rel'], row['tail']), axis=1)

In [ ]:
cui2sty = {}
with open('./data/cui2sty.tsv') as fh:
    for line in fh:
        cui, sty = line.rstrip('\n').split('\t')
        if cui not in cui2sty:
            cui2sty[cui] = sty

In [ ]:
def add_sty(cui):
    if cui in cui2sty:
        return cui2sty[cui]
    else:
        return "NULL"

df['head_sty'] = df.apply(lambda row: add_sty(row['head']), axis=1)
df['tail_sty'] = df.apply(lambda row: add_sty(row['tail']), axis=1)

In [ ]:
triples = {}
with open('./data/12859_2020_3517_MOESM2_ESM.txt') as fh:
    for line in fh:
        _, triple = line.rstrip().split('|')
        sub, rel, obj = triple.split('-')
        triples[(sub, rel.upper(), obj)] = 1

In [ ]:
def is_valid(sub, rel, obj):
    if (sub, rel, obj) in triples:
        return True
    else:
        return False

df['valid'] = df.apply(lambda row: is_valid(row['head_sty'], row['rel'], row['tail_sty']), axis=1)

In [ ]:
df = df[['head', 'head_name', 'head_sty', 'rel', 'tail', 'tail_name', 'tail_sty', 'score', 'novel', 'valid']]

In [ ]:
#df.to_csv('./2020-12-16/DISRUPTS_ComplEx_output.csv', index=False)
#df.to_csv('./2020-12-16/INHIBITS_ComplEx_output.csv', index=False)
#df.to_csv('./2020-12-16/PREVENTS_ComplEx_output.csv', index=False)
#df.to_csv('./2020-12-16/TREATS_ComplEx_output.csv', index=False)

#df.to_csv('./2020-12-16/DISRUPTS_DistMult_output.csv', index=False)
#df.to_csv('./2020-12-16/INHIBITS_DistMult_output.csv', index=False)
#df.to_csv('./2020-12-16/PREVENTS_DistMult_output.csv', index=False)
#df.to_csv('./2020-12-16/TREATS_DistMult_output.csv', index=False)

#df.to_csv('./2020-12-16/DISRUPTS_RotatE_output.csv', index=False)
#df.to_csv('./2020-12-16/INHIBITS_RotatE_output.csv', index=False)
#df.to_csv('./2020-12-16/PREVENTS_RotatE_output.csv', index=False)
#df.to_csv('./2020-12-16/TREATS_RotatE_output.csv', index=False)

In [ ]:
## END OF EXPORT